## 1.1-Importando librerias necesarias.

In [1]:
from gensim.models import KeyedVectors
import matplotlib.pyplot as  plt
from collections import Counter
import pandas as pd
import numpy as np
import random
import string
import math
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

/home/fedricio/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1-Dataset Puestos

In [148]:
df_Jobs = pd.read_csv("Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/Job_Descr/Job_Positions_Scrapped/10_Positions_Only_Qualif.csv")
df_Jobs = df_Jobs.rename(columns={'job_title': 'Job_Title', 'job_description':'Job_Description'})

df_Jobs

,Job_Title,Job_Description
0,Machine Learning Engineer,"BSc/MSc/PhD in Physics, Mathematics, Biomedica..."
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...
2,Data Scientist,"Master’s degree or above in a STEM field, incl..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,..."
4,Full Stack Web Developer,\n\n Graduate Degree in Information Technol...
5,Full Stack Web Developer 2,\n· Enter existing website codebases and exten...
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...
7,HCM Consultant 2,\nConducting Fit/Gap analysis and business /pr...
8,Security Specialist,Work in a fast-paced environment that combine ...
9,Security Specialist 2,\n Handling incoming requests for assistanc...


### 2-Dataset CVs

In [3]:
#Leemos los CVs almacenados en nuestra carpeta y los extraemos uno por uno mediante la libreria PyPDF; devolviendonos
#una secuencia de strings.
import PyPDF2
import os
import collections
from os import listdir
from os.path import isfile, join

pathCVs='Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/PDFs_CVs' #Ruta Relativa, ubicacion de la carpeta.  #Antes, ruta absoluta: r'C:\Users\calon\Desktop\Notebooks\Resume-Scoring-using-NLP-master\Resume-Scoring-using-NLP-master\Resumes'
onlyfiles = [os.path.join(pathCVs, f) for f in os.listdir(pathCVs) if os.path.isfile(os.path.join(pathCVs, f))]

print("Cantidad de CVs extraidos:", len(onlyfiles))

#Funcion para extraer las palabras del CV:
import collections

def pdfextract(file):
    page_content = ""
    pdf_file = open(file, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    c = collections.Counter(range(number_of_pages))
    for i in c:
        page = read_pdf.getPage(i)
        page_content += page.extractText()
    return (page_content.encode('utf-8'))

def extract_text(file):
    text = pdfextract(file).decode('utf-8')
    return text

#Obtenemos todas las palabras del CV sin preprocesar ni nada:
df_Candidates=pd.DataFrame(columns = ['Candidate_Name','Content_CV'])
i=0
while i < len(onlyfiles):
    file=onlyfiles[i]
    base = os.path.basename(file)  #Test_Phoebe Buffay.pdf
    filename = os.path.splitext(base)[0]  #Test_Phoebe Buffay
    dat=extract_text(file)
    data = [{'Candidate_Name':filename, 'Content_CV':dat}]
    df_Candidates=df_Candidates.append(data, ignore_index=True)
    i+=1

df_Candidates

Cantidad de CVs extraidos: 6


,Candidate_Name,Content_CV
0,Test_Chandler,Chandler Bing\n \nchandler.bing@vit.ac.in\n \n...
1,Test_MeghnaLohani,\n \n \n \n \nMeghna Lohani\n \nCampus Addres...
2,Test_AmanSharma,Aman Sharma\n \nCampus Address ...
3,Federico_Calonge_HCM,\n ...
4,Test_VAISHALI BIJOY,VAISHALI BIJOY\n \nVAISHALI.BIJOY2016@vitstude...
5,Test_Phoebe Buffay,Phoebe Buffay\n \n \nPhobe.buffaycat@vit.ac.in...


**Limpieza De los CVs**:
1. Limpieza "general": Pasar todo a minúscula, eliminar signos de puntuación, espacios en blanco.
2. Limpieza de stop words.
3. Steaming para unir palabras: computer_science, machine_learning.
4. Lematización. (VER)
5. Limpieza en txt de datos personales (mails, telefonos).
6. Limpieza de palabras innecesarias en archivo txt (Page, meses, años, etc).

**Limpieza de los Puestos**:
1. Limpieza "general": Pasar todo a minúscula, eliminar signos de puntuación, espacios en blanco.
2. Limpieza de stop words.
3. Steaming para unir palabras: computer_science, machine_learning.
4. Lematización. (VER)
5. Limpieza de palabras innecesarias en archivo txt (VER: por ej. mails que dejan las empresas, telefonos, etc.).
6. Luego, para la obtención de Keywords (VER si es necesario): --> Por el problema que me tomaba las keywords como "you", "I" y demás..., podría hacer un txt con lenguajes y skills por "defecto" y me fijo si están en las keywords... sino no las pongo.


### Funciones para limpiar datos en columnas de un DF

In [149]:
import regex as re #Usado en la función remove_punctuation_and_special_characters.
from nltk.stem import WordNetLemmatizer   #Usado para lematización.
nltk.download('wordnet')
from nltk.tokenize import word_tokenize   #Usado para tokenizar.

#Definimos nuestras stop words desde nltk:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
                
def lower_text(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: x.lower() if isinstance(x,str) else x)
    
def remove_stop_words(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

def remove_punctuation_and_special_characters(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: re.sub('[^\w-]|(\d+)',' ', x) if isinstance(x,str) else x)
                                                                #Macheamos cualquier non-word excepto el -; y tambien macheamos los números (\d+)
    #Macheos:
        #'\W+'                  --> Cualquier non-word.
        #'[()!@#$/,;.:"]'       --> todos esos carácteres especiales que pongo ahi. 
        #'\w+[-]\w+'            --> machea a las palabras non-algo.
        #'[^\w-+]'              --> Machea todas las non-words excepto el guion medio y el +.
        #'\C\+\+'               --> Machea C++.
        #[^\w-+]|(\C\+\+|team)  --> Machea todas las non-words excepto el guion medio y el +. Y tambien machea C++ y la palabra "team"
        #[^\w-+]|(\C\+\+|\d+)   --> Machea todas las non-words excepto el guion medio y el +. Y tambien machea C++ y los números (\d+).
        
    #Para m.sc' --> reemplazar el . por espacio cuando antes y despues del punto esta todo "junto"... para diferenciarlo del punto de una oración. 
    
#Tokenizar es el proceso de parsear Strings de texto en diferentes secciones ("tokens"). Debemos tokenizar teniendo en cuenta qué hacer con las comas, con los espacios en blanco, 
#con los "-", etc. Para esto usamos la función "word_tokenize" que se encargará de esta tarea. 
#Esta funcion NO divide las palabras que estan con un "-"... de esta manera la palabra "scale-up" queda así y NO se divide.
#Tambien tendremos la palabra con una única letra ("c", del lenguaje c) macheado únicamente a un token "c". 
def tokenize_and_lemmatization(text_column):
    tokens = word_tokenize(text_column) #Tokenizamos.
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [wordnet_lemmatizer.lemmatize(tok).lower() for tok in tokens] # Stem words.
    return list(lemmatized_tokens)
    
def cleaning_DF(DF,column_to_clean):
    clean_column='clean_'+column_to_clean
    DF[clean_column]=DF[column_to_clean] #Copiamos el contenido de 'column_to_clean' en 'clean_column' para utilizarla en las funciones posteriores.
    lower_text(DF,clean_column)
    remove_punctuation_and_special_characters(DF,clean_column)
    remove_stop_words(DF,clean_column)
     
def tokenize_and_lemmatize(DF,column_to_clean):
    clean_column='clean_'+column_to_clean
    tokens_column='tokens_'+column_to_clean
    DF[tokens_column] = DF[clean_column].apply(tokenize_and_lemmatization)

[nltk_data] Downloading package wordnet to /home/fedricio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Limpieza de datos del DF de Jobs.

In [150]:
cleaning_DF(df_Jobs,'Job_Description')
tokenize_and_lemmatize(df_Jobs,'Job_Description')
print(df_Jobs['Job_Description'].iloc[1])  #Tomamos el job_description de la posición 1 como ejemplo.
#non-expert  debería mantenerse.
#deberia juntar el Computer Science
print("#######################################################")
print(df_Jobs['clean_Job_Description'].iloc[1]) 
print("#######################################################")
print(df_Jobs['tokens_Job_Description'].iloc[1]) 
print("#######################################################")
df_Jobs

Collaborate with a multidisciplinary team to gain insight into complex biochemical systems.
Design computational models to study various interactions such as interactions between genomes, proteins, and binding sites
Extract various features from the computational models and communicate the results back to the team.
Predict the behaviour of new protein structures on certain binding sites using the computational models.
Work with the team of software engineers to embed your models into production.
Perform other related duties in keeping with the purpose and accountabilities of the job.

M.Sc. or Ph.D. in Engineering/Computer Science, or an equivalent combination of experience and knowledge.
2+ years' experience applying machine learning and deep learning concepts to real-world problems.
Solid programming skills with a focus on writing clean/maintainable code, with 2+ years of experience in Python (preferred), Java, or C++ programming.
Good knowledge of machine learning libraries (Tensorf

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description
0,Machine Learning Engineer,"BSc/MSc/PhD in Physics, Mathematics, Biomedica...",bsc msc phd physics mathematics biomedical sof...,"[bsc, msc, phd, physic, mathematics, biomedica..."
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i..."
2,Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,..."
4,Full Stack Web Developer,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si..."
5,Full Stack Web Developer 2,\n· Enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ..."
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
7,HCM Consultant 2,\nConducting Fit/Gap analysis and business /pr...,conducting fit gap analysis business process m...,"[conducting, fit, gap, analysis, business, pro..."
8,Security Specialist,Work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn..."
9,Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi..."


#### Limpieza de datos del DF de Jobs.

In [7]:
cleaning_DF(df_Candidates,'Content_CV')
tokenize_and_lemmatize(df_Candidates,'Content_CV')

df_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV
0,Test_Chandler,Chandler Bing\n \nchandler.bing@vit.ac.in\n \n...,chandler bing chandler bing vit ac education l...,"[chandler, bing, chandler, bing, vit, ac, educ..."
1,Test_MeghnaLohani,\n \n \n \n \nMeghna Lohani\n \nCampus Addres...,meghna lohani campus address meghna lohani vit...,"[meghna, lohani, campus, address, meghna, loha..."
2,Test_AmanSharma,Aman Sharma\n \nCampus Address ...,aman sharma campus address aman sharma vitstud...,"[aman, sharma, campus, address, aman, sharma, ..."
3,Federico_Calonge_HCM,\n ...,calonge federico matias oracle consultant page...,"[calonge, federico, matias, oracle, consultant..."
4,Test_VAISHALI BIJOY,VAISHALI BIJOY\n \nVAISHALI.BIJOY2016@vitstude...,vaishali bijoy vaishali bijoy vitstudent ac ed...,"[vaishali, bijoy, vaishali, bijoy, vitstudent,..."
5,Test_Phoebe Buffay,Phoebe Buffay\n \n \nPhobe.buffaycat@vit.ac.in...,phoebe buffay phobe buffaycat vit ac education...,"[phoebe, buffay, phobe, buffaycat, vit, ac, ed..."


### AGREGADO: Funciones para tokenizar textos de una columna de un DF

In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import string
#Esto de arriba se descarga LOCALMENTE en home/user/nltk_data.

def tokenize(text):
    #Convierte el texto recibido como entrada en una lista de tokens (palabras).
    #Dentro pasamos todo a minusculas, se eliminan signos de puntuación, espacios en blanco y se eliminan las stop words.
        #input  --> text: el documento como un STRING. 
        #output --> texts_tokens: cada palabra del texto en una LISTA de string. 
    texts_tokens=[]
    for line in text:
        tokens=word_tokenize(line)
        tok=[w.lower() for w in tokens]
        table=str.maketrans('','',string.punctuation)
        strpp=[w.translate(table) for w in tok]
        words=[word for word in strpp if word.isalpha()]
        stop_words=set(stopwords.words('english'))
        words=[w for w in words if not w in stop_words]
        texts_tokens.append(words)
    return texts_tokens

def text_to_vector(text):
    #Convierte el texto del documento en una "term matrix" donde todas las palabras estan listadas junto a 
    #la frecuencia en que aparecen en el texto.
        #input  --> text: el documento como una LISTA de string. 
        #output --> Term matrix: una MATRIZ con cada palabra del documento junto a su frecuencia. 
        
    texts_tokens = tokenize(text)
    #print(texts_tokens)
    return Counter(texts_tokens[0]) #[0] porque es una lista dentro de otra lista.

[nltk_data] Downloading package punkt to /home/fedricio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [141]:
#PARA PROBAR LO DE ARRIBA COMO HACIA ANTES:
#Tomamos como ejemplo la descripción del primer puesto del DF "df_DS_Jobs":
text_description_1st_job = df_Jobs.iloc[1]['Job_Description']
print(text_description_1st_job)

#Usamos la funcion text_to_vector() para text description:
#vector_test_description = text_to_vector([text_description_1st_job])  #Recordar mandar lo de adentro como una LISTA.
#vector_test_description

Counter({'collaborate': 1,
         'multidisciplinary': 1,
         'team': 3,
         'gain': 1,
         'insight': 1,
         'complex': 1,
         'biochemical': 1,
         'systems': 1,
         'design': 2,
         'computational': 3,
         'models': 4,
         'study': 1,
         'various': 2,
         'interactions': 2,
         'genomes': 1,
         'proteins': 1,
         'binding': 2,
         'sites': 2,
         'extract': 1,
         'features': 1,
         'communicate': 1,
         'results': 2,
         'back': 1,
         'predict': 1,
         'behaviour': 1,
         'new': 1,
         'protein': 1,
         'structures': 1,
         'certain': 1,
         'using': 1,
         'work': 1,
         'software': 1,
         'engineers': 1,
         'embed': 1,
         'production': 1,
         'perform': 1,
         'related': 1,
         'duties': 1,
         'keeping': 1,
         'purpose': 1,
         'accountabilities': 1,
         'job': 1,
         '

###  1.3.1.1-Agregando columna 'keywords' a nuestro DF obteniendolas mediante keyword extraction.

Para esto ver previamente el ejemplo general (ejemplo número 5) del Notebook '1-WMD_And_Cosine_Similarity_General'.

#### Resumen: 
Document -> Remove stop words -> Find Term Frequency (TF) -> Find Inverse Document Frequency (IDF) -> Find TF*IDF -> Get top N Keywords

Definimos las funciones principales a utilizar:

In [9]:
from nltk import tokenize
from operator import itemgetter
import math

def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result
    
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))
   
def get_keywords(doc,n):
    from nltk import tokenize
    from operator import itemgetter
    import math

    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize 
    #stop_words = set(stopwords.words('english'))

    total_words = doc.split()
    total_word_length = len(total_words)
    # print(total_word_length)
    
    total_sentences = tokenize.sent_tokenize(doc)
    total_sent_len = len(total_sentences)
    print(total_sentences)
    print(total_sent_len)
    
    #Calculo de TF:
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        #if each_word not in stop_words:
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    
    #Calculo de IDF:
    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        #if each_word not in stop_words:
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    #print(idf_score)

    #Calculo de TF * IDF:
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    # print(tf_idf_score)

    #Obtenemos las 5 palabras top:
    n_palabras_top = get_top_n(tf_idf_score, n)
    # print(n_palabras_top)
    
    #Obtenemos la lista de keywords:
    keyword_list = list(n_palabras_top.keys())   #n_palabras_top es un dic.
    
    return keyword_list

In [19]:
#Ejemplo para comprobar el correcto funcionamiento de las funciones anteriores:
doc_1 = "Computer programming is the process of designing and building an executable computer program to accomplish a specific computing result or to perform a specific task. Programming involves tasks such as: analysis, generating algorithms, profiling algorithms' accuracy and resource consumption, and the implementation of algorithms in a chosen programming language (commonly referred to as coding).[1][2] The source code of a program is written in one or more languages that are intelligible to programmers, rather than machine code, which is directly executed by the central processing unit. The purpose of programming is to find a sequence of instructions that will automate the performance of a task (which can be as complex as an operating system) on a computer, often for solving a given problem. Proficient programming thus often requires expertise in several different subjects, including knowledge of the application domain, specialized algorithms, and formal logic. Tasks accompanying and related to programming include: testing, debugging, source code maintenance, implementation of build systems, and management of derived artifacts, such as the machine code of computer programs. These might be considered part of the programming process, but often the term software development is used for this larger process with the term programming, implementation, or coding reserved for the actual writing of code. Software engineering combines engineering techniques with software development practices. Reverse engineering is a related process used by designers, analysts and programmers to understand and re-create/re-implement."
doc = doc_1.lower()
#'I am a graduate. I want to learn Python. I like learning Python. Python is easy. Python is interesting. Learning increases thinking. Everyone should invest time in learning'
keyword_list = get_keywords(doc,5)  #Le pasamos el número de keywords que queremos y el documento/string.
#dic.keys()
keyword_list

['computer programming is the process of designing and building an executable computer program to accomplish a specific computing result or to perform a specific task.', "programming involves tasks such as: analysis, generating algorithms, profiling algorithms' accuracy and resource consumption, and the implementation of algorithms in a chosen programming language (commonly referred to as coding).", '[1][2] the source code of a program is written in one or more languages that are intelligible to programmers, rather than machine code, which is directly executed by the central processing unit.', 'the purpose of programming is to find a sequence of instructions that will automate the performance of a task (which can be as complex as an operating system) on a computer, often for solving a given problem.', 'proficient programming thus often requires expertise in several different subjects, including knowledge of the application domain, specialized algorithms, and formal logic.', 'tasks acco

['designing', 'building', 'executable', 'accomplish', 'computing']

In [32]:
#Ahora aplicamos la funcion get_keywords(n,doc) a nuestro DF para generar la columna 'keywords':
df_Jobs

,Job_Title,Job_Description
0,Machine Learning Engineer,"BSc/MSc/PhD in Physics, Mathematics, Biomedica..."
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...
2,Data Scientist,"Master’s degree or above in a STEM field, incl..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,..."
4,Full Stack Web Developer,\n\n Graduate Degree in Information Technol...
5,Full Stack Web Developer 2,\n· Enter existing website codebases and exten...
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...
7,HCM Consultant 2,\nConducting Fit/Gap analysis and business /pr...
8,Security Specialist,Work in a fast-paced environment that combine ...
9,Security Specialist 2,\n Handling incoming requests for assistanc...


In [37]:
df_Jobs['Job_Description'] = df_Jobs['Job_Description'].apply(lambda x: x.lower() if isinstance(x,str) else x)


df_Jobs['Keywords'] = df_Jobs['Job_Description'].apply(get_keywords, args=[10])
df_Jobs

,Job_Title,Job_Description,Keywords
0,Machine Learning Engineer,"bsc/msc/phd in physics, mathematics, biomedica...","[(eg,, bsc/msc/phd, physics,, mathematics,, bi..."
1,Machine Learning Engineer 2,collaborate with a multidisciplinary team to g...,"[2+, collaborate, multidisciplinary, gain, ins..."
2,Data Scientist,"master’s degree or above in a stem field, incl...","[master’s, degree, or, above, in, a, stem, fie..."
3,Data Scientist 2,"\nreporting to the director, data & analytics,...","[projects, techniques, skills, organization’s,..."
4,Full Stack Web Developer,\n\n graduate degree in information technol...,"[experience, written, typescript), skills, and..."
5,Full Stack Web Developer 2,\n· enter existing website codebases and exten...,"[5, (required), enter, existing, website, code..."
6,HCM Consultant,\nthe oracle cloud hcm absence consultant will...,"[requirements, experience, work, consulting, c..."
7,HCM Consultant 2,\nconducting fit/gap analysis and business /pr...,"[and, experience, business, management,, abili..."
8,Security Specialist,work in a fast-paced environment that combine ...,"[experience, with, work, fast-paced, combine, ..."
9,Security Specialist 2,\n handling incoming requests for assistanc...,"[security, of, handling, incoming, requests, f..."


In [ ]:
#Transformamos la lista de strings de la columna Keywords en strings separados por ', ':
df_Jobs['Keywords'] = [', '.join(map(str, l)) for l in df_Jobs['Keywords']]

#Minusculas:
df_Jobs['Keywords'] = df_Jobs['Keywords'].apply(lambda x: x.lower() if isinstance(x,str) else x) #Ponemos todas en minusculas (importante para hacer las comparaciones futuras).
df_Jobs['Job_Description'] = df_Jobs['Job_Description'].apply(lambda x: x.lower() if isinstance(x,str) else x) #Tambien en minuscula el Job_Description.

df_Jobs

# VERSION 2 TF IDF.

In [106]:
from nltk import tokenize
from nltk.tokenize import word_tokenize 
from operator import itemgetter
import math

def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result
    
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

def lower_text_2(text):
    new_text = text.lower()
    return new_text

def remove_punctuation_and_special_characters_2(text):
    new_text = re.sub('[^\w-.!?]|(\d+)',' ', text)
    return new_text
    #Todo lo que machee lo reemplaza por un espacio. Machea:
        #1-Todas las non-words excepto el guion medio (-), el punto (.), signo de interrogación final (?) y signo de exclamación final (!). 
        #2-Y los números.

def tokenize_and_lemmatization_2(total_words):
    wordnet_lemmatizer = WordNetLemmatizer()
    text=''
    for each_word in total_words:
        word_lemmatized = wordnet_lemmatizer.lemmatize(each_word).lower()
        text = text+word_lemmatized+' '
    return text

def get_keywords_2(doc,n):
    #Vamos a considerar a nuestro 'doc' como el total de todas nuestras palabras de la celda 'Job Description'.
    #Para calcular el IDF dividimos nuestro 'doc' en 'sentences', donde cada 'sentence' es una oración del la columna 'Job Description'.
    #Para realizar esta separación de oraciones utilizamos la función sent_tokenize (si detecta un punto (.), signo de interrogación final (?) o signo de exclamación final (!) se considera que termina una oración).
    
    # 1-Obtenemos nuestro doc y realizamos la limpieza del mismo:
    doc = remove_punctuation_and_special_characters_2(doc)
    doc = lower_text_2(doc)
    total_words = doc.split()
    doc = tokenize_and_lemmatization_2(total_words)
    print(doc)
    #Realizamos nuevamente el split, ya que algunas palabras cambiaron por la lematización:
    total_words = doc.split()
    total_word_length = len(total_words)
    
    # 2-Realizamos la división por oraciones:
    total_sentences = tokenize.sent_tokenize(doc)
    total_sent_len = len(total_sentences)
    print(total_sentences)
    print(total_sent_len)
    
    #Calculo de TF:
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        each_word = each_word.replace('?','')
        each_word = each_word.replace('!','')
        print(each_word)
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    
    #Calculo de IDF:
    idf_score = {}
    for each_word in total_words:
        #Para eliminar los puntos (.), signos interrogación final (?) y signos de exclamación final (!) de las palabras que expresan fin de oración 
        #(por ejemplo en "Hola Pepe, como estás? Yo bien, gracias." --> reemplazaría el "estás?" por "estás" y el "gracias." por "gracias"). 
        each_word = each_word.replace('.','')
        each_word = each_word.replace('?','')
        each_word = each_word.replace('!','')
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    #print(idf_score)

    #Calculo de TF * IDF:
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    # print(tf_idf_score)

    #Obtenemos las 5 palabras top:
    n_palabras_top = get_top_n(tf_idf_score, n)
    # print(n_palabras_top)
    
    #Obtenemos la lista de keywords:
    keyword_list = list(n_palabras_top.keys())   #n_palabras_top es un dic.
    
    return keyword_list

In [132]:
#Ejemplo para comprobar el correcto funcionamiento de las funciones anteriores:
doc_1 = "Python is fast! Can you believe? ß Computers programming."
#is the process of designing and building an executable computer program to accomplish a specific computing result or to perform a specific task. Programming involves tasks such as: analysis, generating algorithms, profiling algorithms' accuracy and resource consumption, and the implementation of algorithms in a chosen programming language (commonly referred to as coding).[1][2] The source code of a program is written in one or more languages that are intelligible to programmers, rather than machine code, which is directly executed by the central processing unit. The purpose of programming is to find a sequence of instructions that will automate the performance of a task (which can be as complex as an operating system) on a computer, often for solving a given problem. Proficient programming thus often requires expertise in several different subjects, including knowledge of the application domain, specialized algorithms, and formal logic. Tasks accompanying and related to programming include: testing, debugging, source code maintenance, implementation of build systems, and management of derived artifacts, such as the machine code of computer programs. These might be considered part of the programming process, but often the term software development is used for this larger process with the term programming, implementation, or coding reserved for the actual writing of code. Software engineering combines engineering techniques with software development practices. Reverse engineering is a related process used by designers, analysts and programmers to understand and re-create/re-implement."
doc = doc_1.lower()
#'I am a graduate. I want to learn Python. I like learning Python. Python is easy. Python is interesting. Learning increases thinking. Everyone should invest time in learning'
keyword_list = get_keywords_2(doc,5)  #Le pasamos el número de keywords que queremos y el documento/string.
#dic.keys()
keyword_list

python is fast can you believe ß computer programming 
['python is fast can you believe ß', 'computer programming']
2
python
is
fast
can
you
believe
ß
computer
programming


['python', 'is', 'fast', 'can', 'you']

In [120]:
#df_Jobs['Keywords'] = df_Jobs['Job_Description'].apply(get_keywords_2, args=[3])
#df_Jobs

#df_Jobs['Keywords'] = df_Jobs['Job_Description'].apply(get_keywords_2, args=[3])

text_description_1st_job = df_Jobs.iloc[1]['Job_Description']
print(text_description_1st_job)

keyword_list = get_keywords_2(text_description_1st_job,30)  #Le pasamos el número de keywords que queremos y el documento/string.
#dic.keys()
keyword_list

Collaborate with a multidisciplinary team to gain insight into complex biochemical systems.
Design computational models to study various interactions such as interactions between genomes, proteins, and binding sites
Extract various features from the computational models and communicate the results back to the team.
Predict the behaviour of new protein structures on certain binding sites using the computational models.
Work with the team of software engineers to embed your models into production.
Perform other related duties in keeping with the purpose and accountabilities of the job.

M.Sc. or Ph.D. in Engineering/Computer Science, or an equivalent combination of experience and knowledge.
2+ years' experience applying machine learning and deep learning concepts to real-world problems.
Solid programming skills with a focus on writing clean/maintainable code, with 2+ years of experience in Python (preferred), Java, or C++ programming.
Good knowledge of machine learning libraries (Tensorf

['knowledge',
 'of',
 'with',
 'project',
 'experience',
 'collaborate',
 'multidisciplinary',
 'gain',
 'insight',
 'complex',
 'biochemical',
 'systems',
 'study',
 'between',
 'genome',
 'extract',
 'feature',
 'from',
 'communicate',
 'back',
 'predict',
 'behaviour',
 'new',
 'structure',
 'certain',
 'using',
 'models',
 'work',
 'software',
 'engineer']

# VERSION 3 TF-IDF:

Ahora en lugar de considerar al DOC como solo la celda de 'Description', ahora consideramos TODAS las celdas de 'Description'. Asi, cada documento es una celda del job_description.

El problema con la versión 2 es que generalmente en los jobs description no ponen "." para finalizar cada oraciones, entonces me toma todo el texto como 1 sola oración y al ahcer el IDF no tiene sentido porque solo tengo 1 oración.

Cree una variable "text" con TODOS los textos. Y saco el keyword con todos esos textos partiendo en "sentences". Considero como caracter de fin de oración a 'ß'. Y luego si queremos las keywords de 1 job description en particular nos fijamos solo las keywords que aparezcan en este texto (esto lo hacemos en get_top_n).

In [170]:
from nltk import tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktLanguageVars
class BulletPointLangVars(PunktLanguageVars):
    sent_end_chars = ('ß')
tokenizer = PunktSentenceTokenizer(lang_vars = BulletPointLangVars())

from nltk.tokenize import word_tokenize 
from operator import itemgetter
import math

def get_top_n(dict_elem, text_description_1st_job, n):
    # dict_elem.items() son diccionarios con key=palabra y value=valor tf-idf--> ('bsc', 0.0009448441087378112)
    for key in dict_elem.copy():
        if key not in text_description_1st_job:
             #del dict_elem[key]
            dict_elem.pop(key)
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result
    
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

def lower_text_2(text):
    new_text = text.lower()
    return new_text

def remove_punctuation_and_special_characters_2(text):
    new_text = re.sub('[^\w-ß]|(\d+)',' ', text)
    return new_text
    #Todo lo que machee lo reemplaza por un espacio. Machea:
        #1-Todas las non-words excepto el guion medio (-) y el ß.
        #2-Y los números.

def tokenize_and_lemmatization_2(total_words):
    wordnet_lemmatizer = WordNetLemmatizer()
    text=''
    for each_word in total_words:
        word_lemmatized = wordnet_lemmatizer.lemmatize(each_word).lower()
        text = text+word_lemmatized+' '
    return text

def get_keywords_2(doc,text_description_1st_job,n):
    #Vamos a considerar a nuestro 'doc' como el total de todas nuestras palabras de la celda 'Job Description'.
    #Para calcular el IDF dividimos nuestro 'doc' en 'sentences', donde cada 'sentence' es una oración del la columna 'Job Description'.
    #Para realizar esta separación de oraciones utilizamos la función sent_tokenize (si detecta un punto (.), signo de interrogación final (?) o signo de exclamación final (!) se considera que termina una oración).
    
    # 1-Obtenemos nuestro doc y realizamos la limpieza del mismo:
    doc = remove_punctuation_and_special_characters_2(doc)
    doc = lower_text_2(doc)
    total_words = doc.split()
    doc = tokenize_and_lemmatization_2(total_words)
    #print(doc)
    #Realizamos nuevamente el split, ya que algunas palabras cambiaron por la lematización:
    total_words = doc.split()
    total_word_length = len(total_words)
    
    # 2-Realizamos la división por oraciones:
    total_sentences = tokenizer.tokenize(doc)
    total_sent_len = len(total_sentences)
    #print(total_sentences)
    #print(total_sent_len)
    
    #Calculo de TF:
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('ß','')
        #print(each_word)
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    
    #Calculo de IDF:
    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('ß','')
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    #print(idf_score)

    #Calculo de TF * IDF:
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    # print(tf_idf_score)

    #Obtenemos las 5 palabras top:
    n_palabras_top = get_top_n(tf_idf_score, text_description_1st_job, n)
    #print(n_palabras_top)
    
    #Obtenemos la lista de keywords:
    keyword_list = list(n_palabras_top.keys())   #n_palabras_top es un dic.
    
    return keyword_list

In [178]:
text_description_1st_job = df_Jobs.iloc[1]['tokens_Job_Description'] #COMPARAMOS CON ESTA, NO CON JOB_DESCRIPTION YA QUE AHI ESTAN LAS PALABRAS SIN LEMATIZAR ni aplicar ninguna limpieza.


text = ' ß '.join(df_Jobs.Job_Description)
text

#Vamos a obtener las 20 keyword SOLO si estan en el documento 1 (o sea el 1er job description).
keyword_list = get_keywords_2(text,text_description_1st_job,20)  #Le pasamos el número de keywords que queremos y el documento/string.
keyword_list

['multidisciplinary',
 'gain',
 'biochemical',
 'genome',
 'extract',
 'back',
 'predict',
 'behaviour',
 'structure',
 'certain',
 'engineer',
 'embed',
 'keeping',
 'purpose',
 'accountability',
 'job',
 'sc',
 'ph',
 'combination',
 'applying']

### 1.3.2-Dataset CVs.

Los armo extayendo todo el texto de los CVs en inglés ubicados en "Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/PDFs_CVs". 

In [ ]:
#Leemos los CVs almacenados en nuestra carpeta y los extraemos uno por uno mediante la libreria PyPDF; devolviendonos
#una secuencia de strings.
import PyPDF2
import os
import collections
from os import listdir
from os.path import isfile, join

pathCVs='Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/PDFs_CVs' #Ruta Relativa, ubicacion de la carpeta.  #Antes, ruta absoluta: r'C:\Users\calon\Desktop\Notebooks\Resume-Scoring-using-NLP-master\Resume-Scoring-using-NLP-master\Resumes'
onlyfiles = [os.path.join(pathCVs, f) for f in os.listdir(pathCVs) if os.path.isfile(os.path.join(pathCVs, f))]

print("Cantidad de CVs extraidos:", len(onlyfiles))

In [ ]:
#Funcion para extraer las palabras del CV:
import collections

def pdfextract(file):
    page_content = ""
    pdf_file = open(file, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    c = collections.Counter(range(number_of_pages))
    for i in c:
        page = read_pdf.getPage(i)
        page_content += page.extractText()
    return (page_content.encode('utf-8'))

In [ ]:
def extract_text(file):
    text = pdfextract(file).decode('utf-8')
    text = text.replace("\n", "")
    text = text.lower()
    return text

In [ ]:
#Obtenemos todas las palabras del CV sin preprocesar ni nada:
df_Candidates=pd.DataFrame(columns = ['Candidate_Name','Content_CV'])
i=0
while i < len(onlyfiles):
    file=onlyfiles[i]
    base = os.path.basename(file)  #Test_Phoebe Buffay.pdf
    filename = os.path.splitext(base)[0]  #Test_Phoebe Buffay
    dat=extract_text(file)
    data = [{'Candidate_Name':filename, 'Content_CV':dat}]
    df_Candidates=df_Candidates.append(data, ignore_index=True)
    i+=1

df_Candidates

### 1.3.2.1-Limpiando datos.

In [ ]:
df_Candidates.iloc[3]['Content_CV']

In [ ]:
first_content_cv = df_Candidates.iloc[0]['Content_CV']
first_content_cv

## 1.4- Realizando Comparaciones.

### 1.4.1- Usamos el Word2vec descargado.

In [ ]:
#Link del cual descargamos el archivo: https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
EMBEDDING_FILE = '/home/fedricio/Desktop/Embeddings_Utilizados/Word2vec/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

###  1.4.2- Funciones necesarias para Cosine Sim y WDM

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
#Esto de arriba se descarga LOCALMENTE en home/user/nltk_data.

In [ ]:
def tokenize(text):
    #Convierte el texto recibido como entrada en una lista de tokens (palabras).
    #Dentro pasamos todo a minusculas, se eliminan signos de puntuación, espacios en blanco y se eliminan las stop words.
        #input  --> text: el documento como un STRING. 
        #output --> texts_tokens: cada palabra del texto en una LISTA de string. 
    texts_tokens=[]
    for line in text:
        tokens=word_tokenize(line)
        tok=[w.lower() for w in tokens]
        table=str.maketrans('','',string.punctuation)
        strpp=[w.translate(table) for w in tok]
        words=[word for word in strpp if word.isalpha()]
        stop_words=set(stopwords.words('english'))
        words=[w for w in words if not w in stop_words]
        texts_tokens.append(words)
    return texts_tokens

def text_to_vector(text):
    #Convierte el texto del documento en una "term matrix" donde todas las palabras estan listadas junto a 
    #la frecuencia en que aparecen en el texto.
        #input  --> text: el documento como una LISTA de string. 
        #output --> Term matrix: una MATRIZ con cada palabra del documento junto a su frecuencia. 
        
    texts_tokens = tokenize(text)
    #print(texts_tokens)
    return Counter(texts_tokens[0]) #[0] porque es una lista dentro de otra lista.

def get_cosine(doc1, doc2):
    #Get the cosine similarity between two documents.
    #Depends on the angle between two non zero vectors which are constructed by each word frequency in the two documents.
        #input  --> doc1: the first document as STRING.
                    #doc2: the second document as STRING.
        #output --> cosine similarity score

    vec1 = text_to_vector([doc1])
    vec2 = text_to_vector([doc2])
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    
def WMD(doc1,doc2):
      #Preprocess the document first and remove english stopwords then call the function that calculates the word mover distance
      #input --> doc1: the first document as STRING.
                 #doc2: the second document as STRING.
      #output --> Word Mover's Distance score
    #print(doc1)
    first_doc = tokenize([doc1])[0]     #[0] porque es una lista dentro de otra lista.
    #print(first_doc)
    second_doc = tokenize([doc2])[0]    #[0] porque es una lista dentro de otra lista.
    return (word2vec.wmdistance(first_doc, second_doc))  #'wmdistance' return the word mover distance between two documents. 

#### Test para comprobar el funcionamiento de la función text_to_vector

In [ ]:
#Tomamos como ejemplo la descripción del primer puesto del DF "df_DS_Jobs":
text_description_1st_job = df_Jobs.iloc[0]['Job_Description']
text_description_1st_job

In [ ]:
#Usamos la funcion text_to_vector() para text description:
vector_test_description = text_to_vector([text_description_1st_job])  #Recordar mandar lo de adentro como una LISTA.
vector_test_description

###  1.4.3- Comparaciones.

Las comparaciones a realizar son:

    1-Entre el contenido de los CVs y las descripciones de los puestos.
    2-Entre el contenido de los CVs y las keywords de puestos obtenidas mediante Keyword extraction.
    3-Entre el contenido de los CVs y keywords de puestos puestas "a mano" /personalizadas.

#### 1.4.3.1-Entre el contenido de los CVs y las descripciones de los puestos.

In [ ]:
df_Jobs

In [ ]:
df_Candidates

In [ ]:
#Creamos un nuevo DF concatenando los 2 dataframe de manera que se comparen todos los CVs con todos los Jobs:
#Nos quedarán 3 x 6 = 18 filas.
df_Jobs_and_Candidates = pd.merge(df_Candidates.assign(A=1), df_Jobs.assign(A=1), on='A').drop('A', 1)
df_Jobs_and_Candidates

In [ ]:
#Como prueba obtenemos el WMD y Cosine Sim comparando para la 1ra fila (entre las columnas Content_CV y Job_Description)
first_job_descr = df_Jobs_and_Candidates.loc[0,'Job_Description']
first_content_cv = df_Jobs_and_Candidates.loc[0,'Content_CV']

cosine_result = round(get_cosine(first_job_descr,first_content_cv),3)
wmd_result = round(WMD(first_job_descr,first_content_cv),3)
print(cosine_result)
print(wmd_result)
 
similarity_wdm = round((1/(1+wmd_result)),3)  #similarity = 1 / (1 + distance) https://groups.google.com/g/gensim/c/-pRZnsOEaPQ 
print(similarity_wdm)

In [ ]:
#Aplicamos las funciones get_cosine y WMD para TODO el DF... entre el contenido del CV y la descripcion del 
#puesto; y el resultado lo guardamos en  las columnas 'Cosine_Job_Desc' y 'WMD_Job_Desc'.

import time

inicio_cosine = time.time()
df_Jobs_and_Candidates['Cosine_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round(get_cosine(row['Content_CV'],row['Job_Description']),3), axis=1)
fin_cosine = time.time()
print(fin_cosine-inicio_cosine)

inicio_WMD = time.time()
#df_Jobs_and_Candidates['WMD_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round(WMD(row['Content_CV'],row['Job_Description']),3), axis=1)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates['WMD_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round((1/(1+(WMD(row['Content_CV'],row['Job_Description'])))),3), axis=1)
fin_WMD = time.time()
print(fin_WMD-inicio_WMD)

df_Jobs_and_Candidates

#### 1.4.3.2-Entre el contenido de los CVs y las keywords de puestos obtenidas mediante Keyword extraction.

In [ ]:
#APlicamos las funciones get_cosine y WMD entre el contenido del CV y las keywords que obtuvimos previamente 
#del puesto y el resultado lo guardamos en  las columnas 'Cosine_Job_Keywords' y 'WMD_Job_Keywords'.
df_Jobs_and_Candidates['Cosine_Job_Keywords'] = df_Jobs_and_Candidates.apply(lambda row: round(get_cosine(row['Content_CV'],row['Keywords']),3), axis=1)
#df_Jobs_and_Candidates['WMD_Job_Keywords'] = df_Jobs_and_Candidates.apply(lambda row: round(WMD(row['Content_CV'],row['Keywords']),3), axis=1)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates['WMD_Job_Keywords'] = df_Jobs_and_Candidates.apply(lambda row: round((1/(1+(WMD(row['Content_CV'],row['Keywords'])))),3), axis=1)

df_Jobs_and_Candidates

#### 1.4.3.3-Entre el contenido de los CVs y keywords de puestos puestas "a mano" / personalizadas.

In [ ]:
Content_CV_Phoebe_Buffay = df_Jobs_and_Candidates.loc[17,'Content_CV']
print(Content_CV_Phoebe_Buffay)

In [ ]:
#Agregamos 2 filas más al DF con las keywords personalizadas (las segundas keywords no tendrán nada que ver
#al contenido del CV). Y luego aplicamos sim cosine y WDM para estos casos y vemos los resultados.

df_Jobs_and_Candidates.loc[18] = ['Test',Content_CV_Phoebe_Buffay,'','','data science, machine learning, pandas, python, sql','','','',''] 
#Es lo mismo poner las Keywords con o sin comas (da igual el resultado, ya que el procesamiento se encarga de esto). 
df_Jobs_and_Candidates.loc[19] = ['Test',Content_CV_Phoebe_Buffay,'','','football, tennis, spanish','','','',''] 

df_Jobs_and_Candidates.tail()

In [ ]:
#TEST: Usamos la funcion text_to_vector() para las keywords:
vector_test_description = text_to_vector(df_Jobs_and_Candidates.iloc[18]['Keywords'])
vector_test_description

#Ahora si aplicamos sim cosine y WDM para estos casos y vemos los resultados. 
df_Jobs_and_Candidates.loc[18,'Cosine_Job_Keywords'] = round(get_cosine(df_Jobs_and_Candidates.loc[18,'Content_CV'],df_Jobs_and_Candidates.loc[18,'Keywords']),3)

#df_Jobs_and_Candidates.loc[18,'WMD_Job_Keywords'] = round(WMD(df_Jobs_and_Candidates.loc[18,'Content_CV'],df_Jobs_and_Candidates.loc[18,'Keywords']),3)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates.loc[18,'WMD_Job_Keywords'] = round((1/(1+(WMD(df_Jobs_and_Candidates.loc[18,'Content_CV'],df_Jobs_and_Candidates.loc[18,'Keywords'])))),3)

df_Jobs_and_Candidates.loc[19,'Cosine_Job_Keywords'] = round(get_cosine(df_Jobs_and_Candidates.loc[19,'Content_CV'],df_Jobs_and_Candidates.loc[19,'Keywords']),3)

#df_Jobs_and_Candidates.loc[19,'WMD_Job_Keywords'] = round(WMD(df_Jobs_and_Candidates.loc[19,'Content_CV'],df_Jobs_and_Candidates.loc[19,'Keywords']),3)
#APlicando lo de  --> similarity = 1 / (1 + distance):
df_Jobs_and_Candidates.loc[19,'WMD_Job_Keywords'] = round((1/(1+(WMD(df_Jobs_and_Candidates.loc[19,'Content_CV'],df_Jobs_and_Candidates.loc[19,'Keywords'])))),3)

df_Jobs_and_Candidates.tail()

## 1.5- Utilizando glove en lugar de word2vec para aplicar WDM.

In [ ]:
# Load Word Embedding Model
import gensim
from gensim.models.keyedvectors import KeyedVectors

print('gensim version: %s' % gensim.__version__)
#glove_model = gensim.models.KeyedVectors.load_word2vec_format('../model/text/stanford/glove/glove.6B.50d.vec')
glove_model = KeyedVectors.load_word2vec_format('/home/fedricio/Desktop/Embeddings_Utilizados/Glove/glove.6B.50d.txt', binary=False, no_header=True)

In [ ]:
CV_text = df_Jobs_and_Candidates.loc[19,'Content_CV']
CV_text_2 = df_Jobs_and_Candidates.loc[9,'Content_CV']
CV_text_3 = df_Jobs_and_Candidates.loc[12,'Content_CV']
Job_Keyw_text = "data science machine learning pandas python sql"
Job_Keyw_text_2 = "football, tennis, spanish"
texts_to_compare = [Job_Keyw_text,CV_text,CV_text_2,CV_text_3,Job_Keyw_text_2]
texts_tokens = tokenize(texts_to_compare)   #Función definida en las secciones anteriores. 
print(texts_tokens)

In [ ]:
texts_tokens[1]   #CV_text

In [ ]:
#Imprimimos las distancias entre Job_Keyw_text y los 3 CVs:

tokens_job_keyw = texts_tokens[0]
tokens_CV_1 = texts_tokens[1]
tokens_CV_2 = texts_tokens[2]
tokens_CV_3 = texts_tokens[3]
tokens_job_keyw_2 = texts_tokens[4]

distance_1 = glove_model.wmdistance(tokens_job_keyw, tokens_CV_1)
print('Distance with CV_1= %.4f' % distance_1)

distance_2 = glove_model.wmdistance(tokens_job_keyw, tokens_CV_2)
print('Distance with CV_2= %.4f' % distance_2)

distance_3 = glove_model.wmdistance(tokens_job_keyw, tokens_CV_3)
print('Distance with CV_3= %.4f' % distance_3)

#Esta última comparación no debe dar buen score, ya que las keywords no tienen nada que ver con el CV:
distance_4 = glove_model.wmdistance(tokens_job_keyw_2, tokens_CV_1)
print('Distance keywords_2 with CV_1= %.4f' % distance_4)

print('#'*40)
############################################################
#Ahora aplicamos lo de --> similarity = 1 / (1 + distance):

sim_1 = round((1 / (1 + distance_1)),3)
print('Similarity with CV_1= %.4f' % sim_1)

sim_2 = round((1 / (1 + distance_2)),3)
print('Similarity with CV_2= %.4f' % sim_2)

sim_3 = round((1 / (1 + distance_3)),3)
print('Similarity with CV_3= %.4f' % sim_3)

#Esta última comparación no debe dar buen score, ya que las keywords no tienen nada que ver con el CV:
sim_4 = round((1 / (1 + distance_4)),3)
print('Similarity keywords_2 with CV_1= %.4f' % sim_4)

## 1.6- Utilizando spacy en lugar de word2vec o glove para aplicar WDM.

In [ ]:
#Ejemplo de la doc oficial modificado: doc oficial: https://spacy.io/universe/project/wmd-relax
#Para esto descargue spacy: >conda install -c conda-forge spacy
#Y el modelo: python -m spacy download en_core_web_lg
import spacy
import wmd

nlp = spacy.load('en_core_web_lg', create_pipeline=wmd.WMD.create_spacy_pipeline)

In [ ]:
#Ejemplo:
doc1 = nlp("Politician speaks to the media in Illinois.")
doc2 = nlp("The president greets the press in Chicago.")
doc3 = nlp("The president greets the press in Chicago.")
print(doc1.similarity(doc2))
print(doc2.similarity(doc3))

In [ ]:
#Nuestro caso:

#Preprocesamos el doc_1 a comparar (el doc_2 no hace falta porque son keywords):
doc_1 = df_Jobs_and_Candidates.loc[19,'Content_CV']
doc_1_tokenize = tokenize([doc_1])              #Usando la funcion definida en las secciones previas.
new_text_doc_1 = ' '.join(doc_1_tokenize[0])    #Pasamos la lista a string.
new_text_doc_1                                  #Nuevo texto sin espacios, stop words y demas (ver todo lo que hace 'tokenize')

doc1 = nlp(new_text_doc_1)
doc2 = nlp('data science machine learning pandas python sql')
print(doc1.similarity(doc2))

In [ ]:
#Hacemos lo mismo que arriba solo que ahora nuestras keywords serán otras que no tendrán que ver con el CV:

doc_1 = df_Jobs_and_Candidates.loc[19,'Content_CV']
doc_1_tokenize = tokenize([doc_1])              
new_text_doc_1 = ' '.join(doc_1_tokenize[0])    
new_text_doc_1                                 

doc1 = nlp(new_text_doc_1)
doc2 = nlp('football, tennis, spanish')  #CAMBIAMOS ESTO.
print(doc1.similarity(doc2))